In [176]:
!pip install sqlalchemy
!pip install pymysql

In [177]:
import pandas as pd
import mysql.connector
from getpass import getpass

In [178]:
from sqlalchemy import create_engine

In [179]:
# get connected to the sql 

host= '127.0.0.1'
pw= getpass()
user= 'root'

conn = mysql.connector.connect(host= host, user= user, password= pw)

In [180]:
#now are creating a cursor, that will hold our query, execute it, and fetch results

cursor = conn.cursor()

In [181]:
# querying the database to sql

query = """CREATE DATABASE IF NOT EXISTS project;"""

cursor.execute(query)

query1 = "SHOW DATABASES"
cursor.execute(query1)
cursor.fetchall()

[('bank',),
 ('coffeeshop',),
 ('dealership',),
 ('gold',),
 ('information_schema',),
 ('iris',),
 ('mysql',),
 ('performance_schema',),
 ('project',),
 ('sakila',),
 ('students',),
 ('sys',),
 ('website',),
 ('world',)]

In [182]:
#cleaning
demo = pd.read_csv('base_dfs/df_final_demo.csv')
demo

,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0
...,...,...,...,...,...,...,...,...,...
70604,7993686,4.0,56.0,38.5,U,3.0,1411062.68,5.0,5.0
70605,8981690,12.0,148.0,31.0,M,2.0,101867.07,6.0,6.0
70606,333913,16.0,198.0,61.5,F,2.0,40745.00,3.0,3.0
70607,1573142,21.0,255.0,68.0,M,3.0,475114.69,4.0,4.0


In [183]:
nullRows = demo.loc[demo['bal'].isna()]['client_id'].values

In [184]:
demo.dropna(thresh=7, inplace=True)

In [185]:
demo.rename(columns={'gendr': 'gender'}, inplace=True)
demo['clnt_age'].fillna(round(demo['clnt_age'].mean()), inplace=True)

In [186]:
demo['gender'].replace({'X': 'U'}, inplace=True)

In [187]:
#sending to sql
eng = create_engine(f'mysql+pymysql://{user}:{pw}@{host}/project')
demo.to_sql('demo', con= eng)

70595

In [188]:
#cleaning experiment
experiment = pd.read_csv('base_dfs/df_final_experiment_clients.csv')
experiment

,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,NaN
70605,8788427,NaN
70606,266828,NaN
70607,1266421,NaN


In [189]:
list(nullRows)

[7402828,
 355337,
 8412164,
 2222915,
 4876926,
 5277910,
 7616759,
 8191345,
 1227228,
 8611797,
 5144725,
 1037867,
 1618749,
 6021001]

In [190]:
def indexfinder(values, df):
    indexer = []

    for value in values:
        i = df.loc[df['client_id'] == value].index
        indexer.append(i)

    return [x[0] for x in indexer]

In [191]:
indexfinder(nullRows, data)

[709900,
 107345,
 126973,
 110213,
 732475,
 298968,
 110948,
 5812,
 80290,
 362680,
 10869,
 14896,
 297650,
 132349]

In [192]:
experiment.rename(columns={'Variation': 'variation'}, inplace=True)
experiment.fillna('Unknow', inplace=True)
experiment.drop(indexfinder(nullRows, experiment), inplace=True)

In [193]:
experiment

,client_id,variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control
...,...,...
70604,2443347,Unknow
70605,8788427,Unknow
70606,266828,Unknow
70607,1266421,Unknow


In [194]:
#sending to sql
eng2 = create_engine(f'mysql+pymysql://{user}:{pw}@{host}/project')
experiment.to_sql('experiment', con= eng2)


70595

In [195]:
#cleaning data 1
data1 = pd.read_csv('base_dfs\df_final_web_data_pt_1.csv')
data1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [196]:
data1.drop_duplicates(inplace=True)
data1['date_time'] = pd.to_datetime(data1['date_time'])

In [197]:
data1

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [198]:
#cleaning data 2
data2 = pd.read_csv('base_dfs\df_final_web_data_pt_2.csv')
data2

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
412259,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:46:10
412260,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:45:29
412261,9668240,388766751_9038881013,922267647_3096648104_968866,step_1,2017-05-24 18:44:51
412262,9668240,388766751_9038881013,922267647_3096648104_968866,start,2017-05-24 18:44:34


In [199]:
data2.drop_duplicates(inplace=True)
data2['date_time'] = pd.to_datetime(data2['date_time'])

In [200]:
#merging and formating
data = data2.merge(how= 'outer', right= data1)
data

,client_id,visitor_id,visit_id,process_step,date_time
0,763412,601952081_10457207388,397475557_40440946728_419634,confirm,2017-06-06 08:56:00
1,6019349,442094451_91531546617,154620534_35331068705_522317,confirm,2017-06-01 11:59:27
2,6019349,442094451_91531546617,154620534_35331068705_522317,step_3,2017-06-01 11:58:48
3,6019349,442094451_91531546617,154620534_35331068705_522317,step_2,2017-06-01 11:58:08
4,6019349,442094451_91531546617,154620534_35331068705_522317,step_1,2017-06-01 11:57:58
...,...,...,...,...,...
744636,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
744637,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
744638,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
744639,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [201]:
data.sort_values(by='date_time', inplace=True)
data.drop(indexfinder(nullRows, data), inplace=True)
data.reset_index(inplace=True)
data.drop(columns='index', inplace=True)
data

,client_id,visitor_id,visit_id,process_step,date_time
0,9088444,242404224_96732670250,423038079_46067236368_400417,step_3,2017-03-15 00:03:03
1,7179755,167765295_97487764427,264484508_5982901710_928530,start,2017-03-15 00:19:28
2,7179755,167765295_97487764427,264484508_5982901710_928530,step_1,2017-03-15 00:20:50
3,7179755,167765295_97487764427,264484508_5982901710_928530,step_2,2017-03-15 00:22:52
4,7179755,167765295_97487764427,264484508_5982901710_928530,step_3,2017-03-15 00:23:47
...,...,...,...,...,...
744622,4159912,169591677_18042085072,358114812_31313949217_790854,start,2017-06-20 23:58:40
744623,4159912,169591677_18042085072,358114812_31313949217_790854,step_1,2017-06-20 23:59:00
744624,4159912,169591677_18042085072,358114812_31313949217_790854,step_2,2017-06-20 23:59:23
744625,5640210,854970757_75895258102,959123368_90077952691_695747,start,2017-06-20 23:59:53


In [202]:
#sending to sql
eng3 = create_engine(f'mysql+pymysql://{user}:{pw}@{host}/project')
data.to_sql('webdata', con= eng3)

744627